In [2]:
import csv
import json
import pandas as pd
import ast

In [14]:


# Function to convert JSON to CSV
def json_to_csv(json_file, csv_file):
    # Read JSON data
    with open(json_file, mode='r', encoding='utf-8') as f:
        data = json.load(f)
    print(data[0].keys())
    # Write data to CSV file
    with open(csv_file, mode='w', encoding='utf-8', newline='') as f:
        csv_writer = csv.DictWriter(f, fieldnames=data[0].keys())
        csv_writer.writeheader()
        csv_writer.writerows(data)

# Example usage
json_to_csv('mens_shoes.json', 'mens_shoes.csv')


dict_keys(['user_name', 'user_id', 'product_title', 'average_rating', 'total_rating', 'product_category', 'features', 'description', 'product_asin', 'review_title', 'verified_purchase', 'review_rating', 'text', 'helpful_votes', 'total_helpful_votes', 'similarity_text_description', 'similarity_text_features', 'similarity_text_categories', 'sentiment_scores'])


In [15]:
merged_df4 = pd.read_csv('mens_shoes.csv')

In [16]:
# Assuming your original dataframe is named 'merged_df4'

# Calculate number of reviews by each user
reviews_count = merged_df4.groupby('user_id').size().reset_index(name='reviews_count')

# Calculate average length of reviews by each user
merged_df4['review_length'] = merged_df4['text'].apply(lambda x: len(x) if isinstance(x, str) else 1)  # Assuming 'text' column contains review text
# merged_df4['review_length'] = [len(data) for data in merged_df4['text'] if isinstance(data, str) else 1]  # Assuming 'text' column contains review text
# print(type(merged_df4['review_length']))
avg_review_length = merged_df4.groupby('user_id')['review_length'].mean().reset_index(name='avg_review_length')

# Calculate average rating by each user
avg_rating = merged_df4.groupby('user_id')['review_rating'].mean().reset_index(name='avg_rating')

# Calculate average helpfulness votes got by each user on their reviews
avg_helpful_votes = merged_df4.groupby('user_id')['helpful_votes'].mean().reset_index(name='avg_helpful_votes')

# Merge all the calculated features into a single dataframe
user_insights_df1 = reviews_count.merge(avg_review_length, on='user_id').merge(avg_rating, on='user_id').merge(avg_helpful_votes, on='user_id')

# Display the resulting dataframe
# print(user_insights_df1)
user_insights_df1

merged_df4 = pd.merge(merged_df4,user_insights_df1 , on='user_id', how='inner')
merged_df4.head()


,user_name,user_id,product_title,average_rating,total_rating,product_category,features,description,product_asin,review_title,...,total_helpful_votes,similarity_text_description,similarity_text_features,similarity_text_categories,sentiment_scores,review_length,reviews_count,avg_review_length,avg_rating,avg_helpful_votes
0,Doug Darlow,AGZX6S3MRKZNWM54NAOTKUTPO57Q,Cat Footwear Men's Invader Hi Steel Toe Constr...,4.5,468,"Clothing, Shoes & Jewelry, Men, Uniforms, Work...","{'Origin': 'Imported', 'Sole material': 'Rubbe...",Steel toe for tough protection against impact ...,B08S5RCR52,These boots lasted 1 year of intensive use,...,12,0.405520,0.0,0.006384,0.9195,972,1,972.0,5.0,7.0
1,Dollhouse,AGO4T3RH5K6FAAQT6XIP7UXI7WPA,LODAY Waist Trainer Corset for Women Tummy Con...,3.9,44623,"Clothing, Shoes & Jewelry, Women, Clothing, Li...","{'Fabric type': '90% Polyester, 10% Spandex', ...","Double Layer- Comfortable, lightweight stretch...",B0CXPH36F9,Go 2 sizes down,...,120,0.661310,0.0,0.158252,0.3962,289,1,289.0,5.0,0.0
2,C. Stock,AGP4UZURCB4DJSQDFPUEWTCSNC5Q,6pcs Eyeglasses Sunglasses Microfiber Pouch Ba...,4.6,778,"Clothing, Shoes & Jewelry, Men, Accessories, S...","{'Fabric type': 'Microfiber', 'Care instructio...","Package Includes: 6pcs Eyeglasses Pouch, Color...",B099WSY8P2,Doesn't clean the lense,...,54,0.514158,0.0,0.299222,0.7003,223,1,223.0,3.0,0.0
3,Janet W,AH3MZFWW3TXLDWH6CZN635WVGZFA,Lugz Men's Clipper Fashion Sneaker,4.3,32819,"Clothing, Shoes & Jewelry, Men, Shoes, Fashion...","{'Fabric type': '100% Canvas', 'Care instructi...","Cushioned insole for added comfort, Canvas upp...",B08Z75DXHG,They are good looking casual shoes,...,42,0.459030,0.0,0.352875,0.4753,19,1,19.0,5.0,0.0
4,Amazon Customer,AELWM2W6QQEFWG3BX3SOCGKYSJXA,Skechers Men's Waterproof Boot Hiking,4.3,442,"Clothing, Shoes & Jewelry, Men, Shoes, Outdoor...","{'Sole material': 'Rubber', 'Shaft height': 'A...",Take on the trails in rugged style and long-la...,B07N2XZ28F,Comfort and durable.,...,1,0.501387,0.0,0.283064,0.9008,125,1,125.0,5.0,0.0


In [18]:
merged_df4.to_csv('mens_shoes.csv')

In [4]:
# Function to convert CSV to JSON
import csv
import json
def csv_to_json(csv_file, json_file):
    data = []

    # Read CSV file
    with open(csv_file, mode="r", encoding="utf-8") as f:
        csv_reader = csv.DictReader(f)

        # Convert each row into a dictionary and add to the data list
        for row in csv_reader:
            # row["features"] = ast.literal_eval(row["features"])
            row["rating"] = float(row["rating"])
            row["label"] = int(row["label"])
            # row["verified_purchase"] = bool(row["verified_purchase"])
            # row["review_rating"] = float(row["review_rating"])
            # row["helpful_votes"] = int(row["helpful_votes"])
            # # row["total_helpful_votes"] = int(row["total_helpful_votes"])
            # row["similarity_text_description"] = float(
            #     row["similarity_text_description"]
            # )
            # row["similarity_text_features"] = float(row["similarity_text_features"])
            # row["similarity_text_categories"] = float(row["similarity_text_categories"])
            # row["sentiment_scores"] = float(row["sentiment_scores"])
            data.append(row)

    # Write data to JSON file
    with open(json_file, mode="w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


# Example usage
csv_to_json(
    "fake_reviews_dataset.csv",
    "fake_reviews_dataset.json",
)

In [16]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('new_processed_data.csv')

# Check for duplicates based on a specific attribute/column
# For example, if the specific attribute is "email":
# duplicates = df[df.duplicated(subset=['text'], keep=False)]
print(len(df))
duplicates = df.drop_duplicates(subset=['text'], keep='first')

# Print or display the duplicate records
# if not duplicates.empty:
#     print("Duplicate rows based on 'text' column:")
#     print(duplicates)
# else:
#     print("No duplicate rows found based on 'email'.")
print(len(duplicates))
duplicates.to_csv('unique.csv', index=False)

40000
39220
